# Develop a Fake or Real Discriminator

In this notebook, we walk though the steps of developing a machine learning model that is trained to distinguish between fake (=synthetic) and real records. The model's ability to correctly dicriminate between these on an unseen holdout can serve us as another helpful quality criteria for the generated synthetic data. The more realistic those synthetic records are, the harder it will be for any discriminator to tell these apart from the real records.

In order to make the analysis more interesting, we intentionally create synthetic data of lower quality, by limiting the training samples to only a thousand records. Otherwise, the discriminator would not be able to find much signal, if the synthesizer, like MOSTLY AI, is of very high quality.

## Synthesize Data via MOSTLY AI

For this tutorial, we will be using again the UCI Adult Income [[1](#refs)] dataset, which consists of 48842 records across 15 attributes.

1. Download `census.csv` from [here](./census.csv).

2. Synthesize `census.csv` via [MOSTLY AI](https://mostly.ai/), but limit training samples to 1000 records.

<img src='./screen1.png' width="400px"/>

3. Once the job has finished, download the generated synthetic data as CSV file to your computer.

4. Upload the generated synthetic data to this Notebook via executing the next cell.

In [1]:
# upload synthetic dataset
import pandas as pd
try:
    # check whether we are in Google colab
    from google.colab import files
    print("running in COLAB mode")
    repo = 'https://github.com/mostly-ai/mostly-tutorials/raw/dev/train-synthetic-test-real'
    import io
    uploaded = files.upload()
    syn = pd.read_csv(io.BytesIO(list(uploaded.values())[0]))
    print(f"uploaded synthetic data with {syn.shape[0]:,} records and {syn.shape[1]:,} attributes")
except:
    print("running in LOCAL mode")
    repo = '.'
    print("adapt `syn_file_path` to point to your generated synthetic data file")
    syn_file_path = './census-synthetic-1k.csv'
    syn = pd.read_csv(syn_file_path)
    print(f"read synthetic data with {syn.shape[0]:,} records and {syn.shape[1]:,} attributes")

running in LOCAL mode
adapt `syn_file_path` to point to your generated synthetic data file
read synthetic data with 48,842 records and 14 attributes


## Train Discriminator

In [2]:
import lightgbm as lgb
from lightgbm import early_stopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

def prepare_xy(df, target_col, target_val):
    # split target variable `y`
    y = (df[target_col]==target_val).astype(int)
    # convert strings to categoricals, and all others to floats
    str_cols = [col for col in df.select_dtypes(['object', 'string']).columns if col != target_col]
    for col in str_cols:
        df[col] = pd.Categorical(df[col])
    cat_cols = [col for col in df.select_dtypes('category').columns if col != target_col]
    num_cols = [col for col in df.select_dtypes('number').columns if col != target_col]
    for col in num_cols:
        df[col] = df[col].astype('float')
    X = df[cat_cols + num_cols]
    return X, y

def train_model(X, y):
    cat_cols = list(X.select_dtypes('category').columns)
    X_trn, X_val, y_trn, y_val = train_test_split(X, y, test_size=0.2, random_state=1)
    ds_trn = lgb.Dataset(X_trn, label=y_trn, categorical_feature=cat_cols, free_raw_data=False)
    ds_val = lgb.Dataset(X_val, label=y_val, categorical_feature=cat_cols, free_raw_data=False)
    model = lgb.train(
        params={
            'metric': 'auc',  
            'objective': 'binary'
        },
        train_set=ds_trn,
        valid_sets=[ds_val],
        callbacks=[early_stopping(5)],
    )
    return model

import warnings
warnings.filterwarnings('ignore')

In [3]:
# concatenate FAKE and REAL data together
tgt = pd.read_csv(f'{repo}/census.csv')
df = pd.concat([
    tgt.assign(split='REAL'),
    syn.assign(split='FAKE'),
], axis=0)
df.groupby('split').size()

split
FAKE    48842
REAL    48842
dtype: int64

In [4]:
# take a 20% holdout dataset aside for evaluation
trn, hol = train_test_split(df, test_size=0.2, random_state=1)

In [ ]:
%%capture
# train the discriminator on the remaining 80% training dataset
X_trn, y_trn = prepare_xy(trn, 'split', 'FAKE')
model = train_model(X_trn, y_trn)

In [ ]:
# score the model on the holdout dataset, assigning a propability to each record on whether it's FAKE or REAL
X_hol, y_hol = prepare_xy(hol, 'split', 'FAKE')
hol.insert(0, 'is_fake', model.predict(X_hol).round(4))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

auc = roc_auc_score(y_hol, hol.is_fake)
probs_df = pd.concat([
    pd.Series(hol.is_fake, name='probability').reset_index(drop=True),
    pd.Series(y_hol, name='target').reset_index(drop=True)
], axis=1)
fig = sns.displot(data=probs_df, x='probability', hue='target', bins=20, palette=['#008CFB', '#FF004F'])
fig = plt.title(f"Holdout ROC AUC: {auc:.3f}")
plt.show()

As you can see from above chart, the discriminator has learned to pick up some signals that allow it with a varying level of confidence to determine whether a record is FAKE or REAL. 

The AUC can be interpreted as the percentage of cases, where the discriminator is able to correctly spot the FAKE record, given a set of a FAKE and a REAL record.

#### display 5 records that seem very FAKE

In [ ]:
hol.sort_values('is_fake').tail(n=100).sample(n=5, random_state=1)

In these cases, it is the mismatch between `education` and `education_num` that gives away the fact that these are FAKE. In the original data, education level `Assoc-acdm` was mapped to education number 12, whereas in the synthetic data we see various other numeric values.

In [ ]:
pd.crosstab(tgt.education, tgt.education_num)

In [ ]:
pd.crosstab(syn.education, syn.education_num)

#### display 5 records that seem very REAL

I.e. these are type of records, that the synthesizer has apparently failed to create. Thus, as they are then absent from the synthetic data

In [ ]:
hol.sort_values('is_fake').head(n=100).sample(n=5, random_state=1)

## Conclusion

This tutorial has shown how to train a discriminator that is set out to distinguish between RAKE and REAL records. The better the quality of the generated synthetic data, the less likely the discriminator (as well as we humnans) can tell them apart.

## Further exercises

In addition to walking through the above instructions, we suggest..
* measuring the Discriminator's AUC if more training samples are used
* using a different dataset, eg. the UCI bank-marketing dataset [[2](#refs)]
* using a different ML model for the discriminator, eg. a RandomForest model [[3](#refs)]
* using a different synthesizer, eg. SynthCity, SDV, etc.

## References<a class="anchor" name="refs"></a>

1. https://archive.ics.uci.edu/ml/datasets/adult
1. https://archive.ics.uci.edu/ml/datasets/bank+marketing
1. https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html